In [9]:
!python '/content/PatchTST/notebooks/MutiScalePatchTST.py'

Working directory: /content/PatchTST
PyTorch Version: 2.9.0+cu126
CUDA Available: True
Python path head: ['/content/PatchTST/PatchTST_supervised', '/content/PatchTST/PatchTST_physics_integrated', '/content/PatchTST', '/content/PatchTST/notebooks', '/env/python']
NumPy compatibility patch applied for np.Inf -> np.inf
✓ All modules imported successfully
Random seed set to: 2021
Configuration:
  seq_len (look back): 336
  pred_len (forecast): 336
  batch_size: 16
  train_epochs: 15
  enc_in (features): 24
  c_out (output features): 24

Channel groups output all 24 features
Deleted cached file: ./datasets/weather/weather_with_hour.csv
Loading original dataset from: ./datasets/weather/weather.csv
Original shape: (52696, 22)
Original columns: ['date', 'p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)', 'wd (deg)', 'rain (mm)', 'raining (s)', 'SWDR (W/m�)', 'PA

In [ ]:
import sys
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Set repository root path and change to it
repo_root_path = '/content/PatchTST'
os.chdir(repo_root_path)
print(f"Working directory: {os.getcwd()}")

# Build clean sys.path with supervised ahead of physics to avoid utils shadowing
supervised_path = os.path.join(repo_root_path, 'PatchTST_supervised')
physics_path = os.path.join(repo_root_path, 'PatchTST_physics_integrated')
new_paths = [p for p in [supervised_path, physics_path, repo_root_path] if p not in sys.path]
sys.path = new_paths + sys.path  # prepend in desired order

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Python path head: {sys.path[:5]}")

# Numpy fix
if not hasattr(np, 'Inf'):
    np.Inf = np.inf
    np.NaN = np.nan
    np.NAN = np.nan
    np.NINF = np.NINF if hasattr(np, 'NINF') else -np.inf
    print("NumPy compatibility patch applied for np.Inf -> np.inf")
else:
    print("NumPy already has np.Inf attribute")

## 1. Import Physics-Integrated PatchTST Modules

In [ ]:
from MSVLPatchTST.config import PhysicsIntegratedConfig
from MSVLPatchTST.models import PhysicsIntegratedPatchTST
from MSVLPatchTST.training_utils import set_seed, get_target_indices, get_scheduler
from MSVLPatchTST.trainer import train_model
from MSVLPatchTST.evaluation import evaluate_model, evaluate_per_channel
from MSVLPatchTST.data_preprocessing import add_hour_of_day_features

print("✓ All modules imported successfully")

## 2. Load Configuration

In [ ]:
# Create configuration
args = PhysicsIntegratedConfig()
args.random_seed = 2021
set_seed(args.random_seed)

# Override key parameters for comparison with DLinear baseline
args.seq_len = 336       # Look back window (same as DLinear baseline)
args.pred_len = 336      # Prediction horizon (same as DLinear baseline)
args.patience = 3
args.train_epochs = 15
args.batch_size = 16     # Same batch size as DLinear baseline
args.use_cross_channel_encoder = False
args.use_cross_group_attention = True

# CRITICAL: Make model predict ALL 22 features (not just 7 targets)
# Update channel groups to output all indices
for group_name in args.channel_groups:
    # Set output_indices to all indices (0-21 for 22 features)
    args.channel_groups[group_name]['output_indices'] = list(range(22))

print(f"Configuration:")
print(f"  seq_len (look back): {args.seq_len}")
print(f"  pred_len (forecast): {args.pred_len}")
print(f"  batch_size: {args.batch_size}")
print(f"  train_epochs: {args.train_epochs}")
print(f"  enc_in (features): {args.enc_in}")
print(f"  c_out (output features): {args.c_out}")
print(f"\nChannel groups output all 22 features")

## 3. Preprocess Data (Add Hour Features)

In [ ]:
# Add hour-of-day features to dataset
# ALWAYS regenerate from original source - no caching
original_path = os.path.join(args.root_path, 'weather.csv')
enhanced_path = os.path.join(args.root_path, args.data_path)

# Set random seed for reproducibility
np.random.seed(2021)

# ALWAYS delete cached file to force regeneration from original source
if os.path.exists(enhanced_path):
    os.remove(enhanced_path)
    print(f"Deleted cached file: {enhanced_path}")

# Regenerate with NO max pooling
df_enhanced = add_hour_of_day_features(
    original_path, 
    enhanced_path,
    apply_pooling=False  # NO max pooling
)

print(f"\n✓ Data regenerated from original source")
print(f"  Original: {original_path}")
print(f"  Enhanced: {enhanced_path}")
print(f"  Rows: {len(df_enhanced)}, Columns: {len(df_enhanced.columns)}")
print(f"  Max pooling: DISABLED")

## 4. Load Data (Using PatchTST Data Providers)

In [ ]:
# Change to PatchTST_supervised directory for data_provider imports
import importlib

os.chdir(os.path.join(repo_root_path, 'PatchTST_supervised'))
print(f"Changed to: {os.getcwd()}")

# Clear cached modules to avoid stale 'utils' shadowing
for m in [
    'utils', 'utils.timefeatures',
    'data_provider', 'data_provider.data_loader', 'data_provider.data_factory'
]:
    if m in sys.modules:
        sys.modules.pop(m, None)

from data_provider.data_factory import data_provider

os.chdir(repo_root_path)

# Set seed before data loading to ensure reproducible splits
import random
random.seed(2021)
np.random.seed(2021)
torch.manual_seed(2021)

# Create data loaders
train_data, train_loader = data_provider(args, 'train')
val_data, val_loader = data_provider(args, 'val')
test_data, test_loader = data_provider(args, 'test')

print(f"\nData loaded:")
print(f"  Train samples: {len(train_data)}")
print(f"  Val samples: {len(val_data)}")
print(f"  Test samples: {len(test_data)}")

# Verify data normalization parameters match
if hasattr(train_data, 'scaler'):
    print(f"\nData normalization:")
    print(f"  Mean shape: {train_data.scaler.mean_.shape if hasattr(train_data.scaler, 'mean_') else 'N/A'}")
    print(f"  Std shape: {train_data.scaler.scale_.shape if hasattr(train_data.scaler, 'scale_') else 'N/A'}")

In [ ]:
# Verify test data for comparison
print("\n" + "="*60)
print("TEST DATA VERIFICATION (Physics-Integrated)")
print("="*60)

# Get a sample batch to inspect
sample_batch = next(iter(test_loader))
batch_x, batch_y, batch_x_mark, batch_y_mark = sample_batch

print(f"Test loader info:")
print(f"  Total batches: {len(test_loader)}")
print(f"  Batch size: {batch_x.shape[0]}")
print(f"  Input shape (batch_x): {batch_x.shape}")
print(f"  Output shape (batch_y): {batch_y.shape}")
print(f"  Features in test_data: {test_data.data_x.shape[1] if hasattr(test_data, 'data_x') else 'N/A'}")

# Print first sample statistics for verification (FIRST 20 WEATHER FEATURES ONLY)
if hasattr(test_data, 'data_x') and len(test_data.data_x) > 0:
    first_sample_20 = test_data.data_x[0, :20]  # First 20 weather features (indices 0-19, exclude hour_sin, hour_cos at 20-21)
    print(f"\nFirst test sample statistics (First 20 weather features only):")
    print(f"  Mean: {first_sample_20.mean():.6f}")
    print(f"  Std: {first_sample_20.std():.6f}")
    print(f"  Min: {first_sample_20.min():.6f}")
    print(f"  Max: {first_sample_20.max():.6f}")
    print(f"  Sum: {first_sample_20.sum():.6f}")

print("="*60)

In [ ]:
# DIRECT CSV COMPARISON - Verify raw data sources match
import pandas as pd

print("\n" + "="*60)
print("RAW CSV DATA COMPARISON")
print("="*60)

# Load the actual CSV files
original_csv = pd.read_csv(original_path)
enhanced_csv = pd.read_csv(enhanced_path)

# Drop OT from original if present
if 'OT' in original_csv.columns:
    original_csv = original_csv.drop(columns=['OT'])

# For enhanced, we have 20 weather + 2 hour features
# Compare first 20 weather features (columns 1-20, excluding 'date')
original_weather = original_csv.iloc[:, 1:21].values  # Skip 'date' column
enhanced_weather = enhanced_csv.iloc[:, 1:21].values  # Skip 'date' column

print(f"\nOriginal CSV shape: {original_csv.shape}")
print(f"Enhanced CSV shape: {enhanced_csv.shape}")

# Compare a specific row (e.g., row 10000 to avoid boundary effects)
test_row_idx = 10000
original_row = original_weather[test_row_idx]
enhanced_row = enhanced_weather[test_row_idx]

print(f"\nComparing row {test_row_idx} (first 20 weather features):")
print(f"Original: Mean={original_row.mean():.6f}, Std={original_row.std():.6f}, Sum={original_row.sum():.6f}")
print(f"Enhanced: Mean={enhanced_row.mean():.6f}, Std={enhanced_row.std():.6f}, Sum={enhanced_row.sum():.6f}")
print(f"Arrays equal: {np.allclose(original_row, enhanced_row)}")

# Check if all weather features match across entire dataset
weather_match = np.allclose(original_weather, enhanced_weather)
print(f"\nAll weather features match: {weather_match}")

if not weather_match:
    print("WARNING: Weather features differ between original and enhanced CSV!")
    diff_count = np.sum(~np.isclose(original_weather, enhanced_weather))
    print(f"Number of different values: {diff_count} out of {original_weather.size}")

print("="*60)

In [ ]:
# DEEP DATA INSPECTION - Find exact test sample positions
print("\n" + "="*60)
print("DATA NORMALIZATION & SPLIT INSPECTION")
print("="*60)

# Check if scaler exists and display normalization parameters
if hasattr(test_data, 'scaler') and test_data.scaler is not None:
    scaler = test_data.scaler
    print(f"\nNormalization parameters:")
    if hasattr(scaler, 'mean_'):
        print(f"  Mean (first 5): {scaler.mean_[:5]}")
        print(f"  Std (first 5): {scaler.scale_[:5]}")
    print(f"  Scaler type: {type(scaler).__name__}")
else:
    print("\nNo scaler found - data may not be normalized")

# Check train/val/test split information
if hasattr(test_data, 'border1s') and hasattr(test_data, 'border2s'):
    print(f"\nData split borders:")
    print(f"  Test start index (border1s): {test_data.border1s}")
    print(f"  Test end index (border2s): {test_data.border2s}")
    print(f"  Test data length: {test_data.border2s - test_data.border1s}")

# Load the RAW CSV data (before normalization) and find what the test set sees
enhanced_csv_raw = pd.read_csv(enhanced_path)
if 'OT' in enhanced_csv_raw.columns:
    enhanced_csv_raw = enhanced_csv_raw.drop(columns=['OT'])

# Get test data raw values (before normalization)
if hasattr(test_data, 'border1s'):
    test_start_idx = test_data.border1s
    # The first test sample corresponds to this row in the CSV
    first_test_row_idx = test_start_idx
    raw_first_test = enhanced_csv_raw.iloc[first_test_row_idx, 1:21].values  # First 20 weather features
    
    print(f"\nFirst test sample (CSV row {first_test_row_idx}, BEFORE normalization):")
    print(f"  Mean: {raw_first_test.mean():.6f}")
    print(f"  Std: {raw_first_test.std():.6f}")
    print(f"  Sum: {raw_first_test.sum():.6f}")
    
    # Now check the normalized version
    if hasattr(test_data, 'data_x') and len(test_data.data_x) > 0:
        normalized_first_test = test_data.data_x[0, :20]
        print(f"\nFirst test sample (AFTER normalization):")
        print(f"  Mean: {normalized_first_test.mean():.6f}")
        print(f"  Std: {normalized_first_test.std():.6f}")
        print(f"  Sum: {normalized_first_test.sum():.6f}")

print("="*60)

In [ ]:
# Visualize ground truth for first test sample
%matplotlib inline
import matplotlib.pyplot as plt

# Features to visualize
features_to_plot = [
    'p (mbar)',           # air pressure
    'T (degC)',           # temperature
    'wv (m/s)',           # wind speed
    'max. wv (m/s)',      # maximum wind speed
    'rain (mm)',          # rainfall amount
    'raining (s)'         # rainfall duration
]

# All 22 feature names (20 weather + 2 hour)
all_feature_names = [
    'p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 
    'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 
    'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)', 
    'wd (deg)', 'rain (mm)', 'raining (s)', 'SWDR (W/m²)', 
    'PAR (µmol/m²/s)', 'max. PAR (µmol/m²/s)', 'Tlog (degC)', 'hour_sin', 'hour_cos'
]

# Get first test sample (using same seed as inference for consistency)
np.random.seed(2021)
if hasattr(test_data, 'data_x') and len(test_data.data_x) > 0:
    # Get the first sample's ground truth sequence
    first_sample = test_data.data_x[0]  # Shape: (22,) - single timestep
    
    # For visualization, we'll show a longer sequence - get from batch
    sample_gt = batch_y[0].cpu().numpy()  # Shape: (seq_len, 22)
    
    print(f"\nVisualizing ground truth from test data")
    print(f"Ground truth shape: {sample_gt.shape}")
    
    # Create figure
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    # Plot each feature
    for i, feature_name in enumerate(features_to_plot):
        feature_idx = all_feature_names.index(feature_name)
        ax = axes[i]
        
        time_steps = np.arange(len(sample_gt))
        ax.plot(time_steps, sample_gt[:, feature_idx], 
                linewidth=2, color='blue', alpha=0.8)
        
        ax.set_title(feature_name, fontsize=12, fontweight='bold')
        ax.set_xlabel('Time Step', fontsize=10)
        ax.set_ylabel('Value', fontsize=10)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('Test Data Ground Truth - First Sample', 
                 fontsize=14, fontweight='bold', y=1.00)
    plt.show()
    
    print(f"✓ Ground truth visualization complete")
else:
    print("Test data not available for visualization")

## 5. Create Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create model
model = PhysicsIntegratedPatchTST(args).float().to(device)

# Get target indices and names
target_indices, target_names = get_target_indices(args.channel_groups)

print(f"\nModel created:")
print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print("  Target variables by group:")
for group, info in args.channel_groups.items():
    output_indices = set(info.get('output_indices', []))
    src_names = info.get('names', [])
    tgt_names = [src_names[i] for i, idx in enumerate(info['indices']) if idx in output_indices] if src_names else []
    print(f"    {group}: {', '.join(tgt_names) if tgt_names else '(names not provided)'}")

In [ ]:
# Inspect full model architecture
print("\nModel architecture:\n")
print(model)


## 6. Setup Training

In [ ]:
# Create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=args.learning_rate, weight_decay=1e-4)

# Create scheduler (OneCycleLR as in baseline notebook)
train_steps = len(train_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    steps_per_epoch=train_steps,
    pct_start=args.pct_start,
    epochs=args.train_epochs,
    max_lr=args.learning_rate
)

# Create loss function
criterion = nn.MSELoss()

# Create checkpoint directory
checkpoint_path = args.checkpoints
os.makedirs(checkpoint_path, exist_ok=True)

print("Training setup complete")

## 7. Train Model

In [ ]:
import time

# Train the model
start_time = time.time()
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    criterion=criterion,
    args=args,
    device=device,
    target_indices=target_indices,
    checkpoint_path=checkpoint_path
)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

## 8. Plot Training History

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Overall losses
axes[0].plot(history['train_losses'], label='Train Loss', marker='o', markersize=3)
axes[0].plot(history['val_losses'], label='Validation Loss', marker='s', markersize=3)
axes[0].plot(history['test_losses'], label='Test Loss', marker='^', markersize=3)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training History')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Target variable losses
for target_name, losses in history['target_variable_losses'].items():
    axes[1].plot(losses, label=target_name.capitalize(), marker='o', markersize=3)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MSE Loss')
axes[1].set_title('Target Variable Losses')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Load Best Checkpoint and Evaluate MSE/MAE

In [ ]:
# Load best checkpoint for inference
checkpoint_file = os.path.join(checkpoint_path, 'checkpoint.pth')
print(f"Loading checkpoint from: {checkpoint_file}")

# The checkpoint is saved as a direct state_dict
checkpoint = torch.load(checkpoint_file, map_location=device, weights_only=False)
model.load_state_dict(checkpoint)
model.eval()
print(f"✓ Checkpoint loaded successfully")

# Run inference on test set
preds = []
trues = []

with torch.no_grad():
    for batch_x, batch_y, batch_x_mark, batch_y_mark in test_loader:
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        
        # Model forward takes only x (input sequence)
        outputs = model(batch_x)
        preds.append(outputs.cpu().numpy())
        
        # Extract only the last pred_len timesteps from batch_y
        batch_y_target = batch_y[:, -args.pred_len:, :]
        trues.append(batch_y_target.cpu().numpy())

preds = np.concatenate(preds, axis=0)
trues = np.concatenate(trues, axis=0)
print(f"\n✓ Inference complete")
print(f"  Predictions shape: {preds.shape}")
print(f"  Ground truth shape: {trues.shape}")

# Calculate metrics on TARGET features only
preds_target = preds[:, :, target_indices]
trues_target = trues[:, :, target_indices]

# Convert to torch tensors and compute MSE (same as training loss)
preds_torch = torch.from_numpy(preds_target).float()
trues_torch = torch.from_numpy(trues_target).float()
mse_criterion = nn.MSELoss()
mse = mse_criterion(preds_torch, trues_torch).item()
mae = np.mean(np.abs(preds_target - trues_target))

print(f"\n{'='*50}")
print(f"EVALUATION RESULTS (Target Features Only)")
print(f"{'='*50}")
print(f"  Test Samples: {len(preds)}")
print(f"  MSE: {mse:.6f}")
print(f"  MAE: {mae:.6f}")
print(f"  RMSE: {np.sqrt(mse):.6f}")
print(f"{'='*50}")

In [ ]:
# Debug: Check what the model actually outputs
print("\n" + "="*60)
print("MODEL OUTPUT VERIFICATION")
print("="*60)
print(f"Model c_out (expected output channels): {args.c_out}")
print(f"Target indices from config: {target_indices}")
print(f"Target names from config: {target_names}")
print(f"\nActual predictions shape: {preds.shape}")
print(f"Actual ground truth shape: {trues.shape}")
print(f"\nMismatch detected!" if preds.shape[2] != trues.shape[2] else "Shapes match")
print("="*60)

## 10. Visualize Predictions

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Features to plot
features_to_plot = [
    'p (mbar)',           # air pressure
    'T (degC)',           # temperature
    'wv (m/s)',           # wind speed
    'max. wv (m/s)',      # maximum wind speed
    'rain (mm)',          # rainfall amount
    'raining (s)'         # rainfall duration
]

# Full 22-feature names (22 features in ground truth: 20 weather + 2 hour)
full_feature_names = [
    'p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 
    'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 
    'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)', 
    'wd (deg)', 'rain (mm)', 'raining (s)', 'SWDR (W/m²)', 
    'PAR (µmol/m²/s)', 'max. PAR (µmol/m²/s)', 'Tlog (degC)', 'hour_sin', 'hour_cos'
]

# Model only predicts these 7 target features at indices: [0, 1, 11, 12, 14, 15]
# Map predicted feature indices to their position in the 7-feature prediction array
pred_feature_map = {
    0: 0,   # p (mbar) -> pred index 0
    1: 1,   # T (degC) -> pred index 1  
    11: 2,  # wv (m/s) -> pred index 2
    12: 3,  # max. wv (m/s) -> pred index 3
    14: 4,  # rain (mm) -> pred index 4
    15: 5   # raining (s) -> pred index 5
}

# Select a random sample using seed 2021
np.random.seed(2021)
sample_idx = np.random.randint(0, len(preds))

# Get sample prediction (7 features) and ground truth (22 features)
sample_pred = preds[sample_idx]  # Shape: (pred_len, 7)
sample_true_full = trues[sample_idx]  # Shape: (pred_len, 22)

print(f"Sample {sample_idx} shapes: pred={sample_pred.shape}, true={sample_true_full.shape}")
print(f"Target indices (in 22-feature space): {target_indices}")

# Create figure
fig, axes = plt.subplots(1, len(features_to_plot), figsize=(24, 4))

# Plot each feature
for i, feature_name in enumerate(features_to_plot):
    # Get index in the full 22-feature ground truth
    gt_idx = full_feature_names.index(feature_name)
    
    # Get index in the 7-feature prediction array
    if gt_idx in pred_feature_map:
        pred_idx = pred_feature_map[gt_idx]
        
        ax = axes[i]
        time_steps = np.arange(len(sample_pred))
        
        # Plot ground truth from full 22-feature array
        ax.plot(time_steps, sample_true_full[:, gt_idx], label='Ground Truth', 
                linewidth=2, color='blue', alpha=0.7)
        
        # Plot prediction from 7-feature prediction array
        ax.plot(time_steps, sample_pred[:, pred_idx], label='Prediction', 
                linewidth=2, color='red', alpha=0.7, linestyle='--')
        
        ax.set_title(feature_name, fontsize=12, fontweight='bold')
        ax.set_xlabel('Time Step', fontsize=10)
        ax.set_ylabel('Value', fontsize=10)
        ax.legend(loc='best', fontsize=8)
        ax.grid(True, alpha=0.3)
    else:
        print(f"Warning: {feature_name} not in predicted features")

plt.tight_layout()
plt.suptitle(f'Weather Forecast Predictions (Sample {sample_idx}, Horizon: {args.pred_len} steps)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.show()

print(f'\nPlotted sample index: {sample_idx}')